In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./submissions.csv')
data = data.dropna()
data = data.astype({'execution_time': 'int32'})

In [3]:
print(data.size)

113507010


In [4]:
users = data['user_id']
unique_users = users.unique()

In [5]:
data_ac = data[data['result'] == 'AC']

In [6]:
data_ac.head()

,id,epoch_second,problem_id,contest_id,user_id,language,point,length,result,execution_time
0,11777888,1586677338,abc151_c,abc151,i_lohas_MATCH,C++14 (GCC 5.4.1),300.0,777,AC,73
1,10100114,1581749288,abc141_b,abc141,rniya,C++14 (GCC 5.4.1),200.0,1002,AC,1
2,1434499,1500261926,abc067_b,abc067,naoki2016,C++14 (GCC 5.4.1),200.0,598,AC,1
4,11168255,1585009689,dwango2017qual_a,dwacon2017-prelims,t_kato,PyPy3 (2.4.0),100.0,99,AC,172
5,11197219,1585128140,agc006_b,agc006,zoooma13,C++14 (GCC 5.4.1),400.0,651,AC,38


In [7]:
list_for_exec = ['problem_id', 'language', 'execution_time']
list_for_leng =  ['problem_id', 'language', 'length']
data_exec = data_ac[list_for_exec]
data_leng = data_ac[list_for_leng]

In [8]:
group_prob_lang_exec = data_exec.groupby(['language', 'problem_id'])
group_prob_lang_leng = data_leng.groupby(['language', 'problem_id'])

In [9]:
exec_statistics = group_prob_lang_exec.min().rename(columns={'execution_time': 'min'})

In [10]:
exec_statistics_10 =  group_prob_lang_exec.quantile(0.1, interpolation='higher')
exec_statistics_25 =  group_prob_lang_exec.quantile(0.25, interpolation='higher')
exec_statistics_50 =  group_prob_lang_exec.quantile(0.50, interpolation='higher')
exec_statistics_75 =  group_prob_lang_exec.quantile(0.75, interpolation='higher')

In [11]:
exec_statistics['10%'] = exec_statistics_10['execution_time']
exec_statistics['25%'] = exec_statistics_25['execution_time']
exec_statistics['50%'] = exec_statistics_50['execution_time']
exec_statistics['75%'] = exec_statistics_75['execution_time']

In [12]:
exec_statistics.head()

min  10%  25%  50%  75%
language             problem_id                         
Ada2012 (GNAT 9.2.1) abc162_a      4    4    4    4    4
                     abc164_a      2    5    5    5    5
                     abc164_b      3    3    3    3    3
                     abc165_a      4    4    4    4    4
Awk (GNU Awk 4.1.4)  abc162_a      3    3    4    8   12

In [13]:
length_statistics =  group_prob_lang_leng.min().rename(columns={'length': 'min'})

In [14]:
length_statistics_10 =  group_prob_lang_leng.quantile(0.1, interpolation='higher')
length_statistics_25 =  group_prob_lang_leng.quantile(0.25, interpolation='higher')
length_statistics_50 =  group_prob_lang_leng.quantile(0.50, interpolation='higher')
length_statistics_75 =  group_prob_lang_leng.quantile(0.75, interpolation='higher')

In [15]:
length_statistics['10%'] = length_statistics_10['length']
length_statistics['25%'] = length_statistics_25['length']
length_statistics['50%'] = length_statistics_50['length']
length_statistics['75%'] = length_statistics_75['length']

In [16]:
length_statistics.head()

min  10%  25%  50%  75%
language             problem_id                         
Ada2012 (GNAT 9.2.1) abc162_a    438  438  438  438  438
                     abc164_a    280  299  299  299  299
                     abc164_b    483  483  483  483  483
                     abc165_a    423  423  423  423  423
Awk (GNU Awk 4.1.4)  abc162_a     17   17   17   19   20

In [17]:
list_for_user_info = ['user_id', 'problem_id', 'language', 'length', 'execution_time']
pre_user_info = data_ac[list_for_user_info]

In [18]:
pre_user_info.head()

,user_id,problem_id,language,length,execution_time
0,i_lohas_MATCH,abc151_c,C++14 (GCC 5.4.1),777,73
1,rniya,abc141_b,C++14 (GCC 5.4.1),1002,1
2,naoki2016,abc067_b,C++14 (GCC 5.4.1),598,1
4,t_kato,dwango2017qual_a,PyPy3 (2.4.0),99,172
5,zoooma13,agc006_b,C++14 (GCC 5.4.1),651,38


In [19]:
group_user_lang_prob = pre_user_info.groupby(['user_id', 'language', 'problem_id'], as_index=False)

In [20]:
user_info = group_user_lang_prob.min()

In [21]:
user_info.size

27562045

In [22]:
def get_score(value, border):
    if value <= border['10%']:
        return 5
    
    elif value <= border['25%']:
        return 4
    
    elif value <= border['50%']:
        return 3
    
    elif value <= border['75%']:
        return 2
    
    else:
        return 1

In [23]:
length_scores = []
exec_scores = []

for idx, row in user_info.iterrows():
    language = row['language']
    problem_id = row['problem_id']
    length = row['length']
    exec_time = row['execution_time']
    
    length_border = length_statistics.loc[language, problem_id]
    exec_border = exec_statistics.loc[language, problem_id]
    
    length_score = get_score(length, length_border)
    exec_score = get_score(exec_time, exec_border)
    
    length_scores.append(length_score)
    exec_scores.append(exec_score)    

In [24]:
user_info['length_score'] = length_scores

In [25]:
user_info['exec_score'] = exec_scores

In [26]:
user_info.head()

,user_id,language,problem_id,length,execution_time,length_score,exec_score
0,0071717,Python3 (3.4.3),abc161_b,704,17,1,5
1,011337,C++ (GCC 9.2.1),abc162_a,203,2,5,5
2,011337,C++ (GCC 9.2.1),abc162_b,204,7,5,2
3,011337,C++ (GCC 9.2.1),abc162_c,386,346,4,2
4,011337,C++ (GCC 9.2.1),abc164_a,154,7,5,1


In [27]:
user_status = user_info.loc[:, ['user_id', 'language', 'length_score', 'exec_score']]

In [28]:
user_status.head()

,user_id,language,length_score,exec_score
0,0071717,Python3 (3.4.3),1,5
1,011337,C++ (GCC 9.2.1),5,5
2,011337,C++ (GCC 9.2.1),5,2
3,011337,C++ (GCC 9.2.1),4,2
4,011337,C++ (GCC 9.2.1),5,1


In [29]:
user_scores = user_status.groupby(['user_id', 'language'], as_index=False).sum()
user_ac_count = user_status.groupby(['user_id', 'language'], as_index=False).count()

In [30]:
user_scores['ac_count'] = user_ac_count['length_score'].values

In [31]:
user_scores['length_ave'] = user_scores['length_score'] / user_scores['ac_count']
user_scores['exec_ave'] = user_scores['exec_score'] / user_scores['ac_count']

In [32]:
user_scores.head()

,user_id,language,length_score,exec_score,ac_count,length_ave,exec_ave
0,0071717,Python3 (3.4.3),1,5,1,1.000000,5.000000
1,011337,C++ (GCC 9.2.1),34,22,7,4.857143,3.142857
2,011337,C++14 (GCC 5.4.1),34,36,8,4.250000,4.500000
3,04089,C (GCC 9.2.1),5,3,2,2.500000,1.500000
4,04089,C++ (GCC 9.2.1),9,4,2,4.500000,2.000000


In [33]:
filterd_user_scores = user_scores[user_scores['ac_count'] >= 50]

In [34]:
filterd_user_scores.head()

,user_id,language,length_score,exec_score,ac_count,length_ave,exec_ave
41,300iq,C++14 (GCC 5.4.1),355,581,193,1.839378,3.010363
77,A0ikun_1818,C (GCC 5.4.1),203,438,126,1.611111,3.476190
83,A0iro,C++14 (GCC 5.4.1),881,1069,284,3.102113,3.764085
99,A1phamath,C++14 (GCC 5.4.1),119,291,67,1.776119,4.343284
101,A1phamath,PyPy3 (2.4.0),188,261,137,1.372263,1.905109


In [73]:
filter_lang = filterd_user_scores['language'] ==  'C++14 (GCC 5.4.1)'

In [74]:
lang_ranking = filterd_user_scores[filter_lang]

In [75]:
lang_ranking.head()

,user_id,language,length_score,exec_score,ac_count,length_ave,exec_ave
41,300iq,C++14 (GCC 5.4.1),355,581,193,1.839378,3.010363
83,A0iro,C++14 (GCC 5.4.1),881,1069,284,3.102113,3.764085
99,A1phamath,C++14 (GCC 5.4.1),119,291,67,1.776119,4.343284
118,A7075,C++14 (GCC 5.4.1),537,601,136,3.948529,4.419118
163,AC961009,C++14 (GCC 5.4.1),3840,4111,980,3.918367,4.194898


In [76]:
exec_score_ranking = lang_ranking.sort_values('exec_score', ascending=False).reset_index()
exec_score_ranking.index = exec_score_ranking.index + 1

In [77]:
list_for_exec_rank = ['user_id', 'language', 'exec_score', 'exec_ave', 'ac_count']

In [78]:
exec_score_ranking[list_for_exec_rank].head(20)

,user_id,language,exec_score,exec_ave,ac_count
1,tozangezan,C++14 (GCC 5.4.1),9161,3.888370,2356
2,olphe,C++14 (GCC 5.4.1),8599,3.613025,2380
3,ha15,C++14 (GCC 5.4.1),7998,3.607578,2217
4,kmjp,C++14 (GCC 5.4.1),7080,3.268698,2166
5,beet,C++14 (GCC 5.4.1),7039,2.991500,2353
6,Rubikun,C++14 (GCC 5.4.1),6986,3.384690,2064
7,fumiphys,C++14 (GCC 5.4.1),6859,3.608101,1901
8,Pulmn,C++14 (GCC 5.4.1),6780,2.928726,2315
9,risujiroh,C++14 (GCC 5.4.1),6561,3.863958,1698
10,ei13333,C++14 (GCC 5.4.1),6500,3.134041,2074


In [79]:
exec_ave_ranking = lang_ranking.sort_values('exec_ave', ascending=False).reset_index()
exec_ave_ranking.index = exec_ave_ranking.index + 1

In [80]:
exec_ave_ranking[list_for_exec_rank].head(20)

,user_id,language,exec_score,exec_ave,ac_count
1,monamochi,C++14 (GCC 5.4.1),375,5.000000,75
2,downe,C++14 (GCC 5.4.1),255,5.000000,51
3,bun_mhib,C++14 (GCC 5.4.1),265,5.000000,53
4,YmuuumT,C++14 (GCC 5.4.1),275,5.000000,55
5,hyoutann,C++14 (GCC 5.4.1),260,5.000000,52
6,shunring,C++14 (GCC 5.4.1),578,4.982759,116
7,MASSA,C++14 (GCC 5.4.1),433,4.977011,87
8,youmu73,C++14 (GCC 5.4.1),646,4.969231,130
9,momig,C++14 (GCC 5.4.1),649,4.954198,131
10,httptaniyan,C++14 (GCC 5.4.1),480,4.948454,97


In [68]:
me = exec_ave_ranking['user_id'] == 'parsely'
exec_ave_ranking[me]

,index,user_id,language,length_score,exec_score,ac_count,length_ave,exec_ave
1809,135828,parsely,Python3 (3.4.3),1332,1923,521,2.556622,3.690979


In [59]:
list_for_length_rank = ['user_id', 'language', 'length_score', 'length_ave', 'ac_count']

In [82]:
length_score_ranking = lang_ranking.sort_values('length_score', ascending=False).reset_index()
length_score_ranking.index = length_score_ranking.index + 1

In [ ]:
length_score_ranking[list_for_length_rank].head(20)

In [ ]:
length_ave_ranking = lang_ranking.sort_values('length_ave', ascending=False).reset_index()
length_ave_ranking.index = length_ave_ranking.index + 1

In [ ]:
length_ave_ranking[list_for_length_rank].head(20)

In [ ]:
lang_ranking[lang_ranking['user_id'] == 'parsely']

In [83]:
lang_ranking[lang_ranking['user_id'] == 'parsely']

,user_id,language,length_score,length_ave,ac_count
1,kotatsugame,C++14 (GCC 5.4.1),8197,4.813271,1703
2,ha15,C++14 (GCC 5.4.1),8055,3.633288,2217
3,beet,C++14 (GCC 5.4.1),8041,3.417340,2353
4,yuma220284,C++14 (GCC 5.4.1),7931,4.401221,1802
5,E869120,C++14 (GCC 5.4.1),7367,3.694584,1994
6,leaf1415,C++14 (GCC 5.4.1),7265,3.687817,1970
7,autumn_eel,C++14 (GCC 5.4.1),6824,4.108368,1661
8,Rubikun,C++14 (GCC 5.4.1),6749,3.269864,2064
9,ei13333,C++14 (GCC 5.4.1),6538,3.152363,2074
10,drogskol,C++14 (GCC 5.4.1),6509,4.137953,1573


In [84]:
length_ave_ranking = lang_ranking.sort_values('length_ave', ascending=False).reset_index()
length_ave_ranking.index = length_ave_ranking.index + 1

In [85]:
length_ave_ranking[list_for_length_rank].head(20)

,user_id,language,length_score,length_ave,ac_count
1,Hohn,C++14 (GCC 5.4.1),298,4.966667,60
2,t1234,C++14 (GCC 5.4.1),4668,4.965957,940
3,woon813,C++14 (GCC 5.4.1),296,4.933333,60
4,aufeas,C++14 (GCC 5.4.1),515,4.904762,105
5,bennettz,C++14 (GCC 5.4.1),990,4.900990,202
6,GEHOx2,C++14 (GCC 5.4.1),1050,4.883721,215
7,narcolepsia,C++14 (GCC 5.4.1),1115,4.868996,229
8,rapier,C++14 (GCC 5.4.1),467,4.864583,96
9,BackOnTrack,C++14 (GCC 5.4.1),710,4.863014,146
10,gero,C++14 (GCC 5.4.1),908,4.855615,187


In [45]:
lang_ranking[lang_ranking['user_id'] == 'parsely']

,user_id,language,length_score,exec_score,ac_count,length_ave,exec_ave
135828,parsely,Python3 (3.4.3),1332,1923,521,2.556622,3.690979
